# NX-MIMOSA Quick Start — Multi-Target Tracking in 5 Minutes

**Adaptive Multi-Sensor Multi-Target Radar Tracker**

This notebook demonstrates:
1. Creating a tracker with domain presets
2. Processing simulated radar scans
3. Visualizing tracks
4. Using all 3 data association methods (GNN, JPDA, MHT)
5. Computing NATO-standard quality metrics

In [ ]:
import numpy as np
import sys; sys.path.insert(0, '..')
from nx_mimosa import (
    MultiTargetTracker, compute_ospa, compute_siap_metrics,
    AssociationMethod
)
print("NX-MIMOSA imported successfully")

## 1. Create a Tracker — One Line

In [ ]:
# Military domain preset: auto-configures for 9g fighters, SAMs, etc.
tracker = MultiTargetTracker(dt=1.0, r_std=50.0, domain="military")
print(f"Association: {tracker.association}")
print(f"Tracks: {len(tracker.tracks)}")

## 2. Simulate 3 Targets — Different Flight Profiles

In [ ]:
np.random.seed(42)

# Target ground truth generators
def fighter_trajectory(t):
    """Fighter: 250 m/s, mild turn"""
    x = 250 * t
    y = 5000 + 200 * np.sin(t * 0.1)
    z = 8000 + 50 * np.cos(t * 0.05)
    return np.array([x, y, z])

def transport_trajectory(t):
    """Transport aircraft: 200 m/s, straight and level"""
    return np.array([200 * t + 10000, -100 * t + 20000, 11000.0])

def missile_trajectory(t):
    """SAM missile: 800 m/s, climbing"""
    return np.array([800 * t, 1000 + 300 * t, 500 + 400 * t])

n_scans = 60
truths = []  # For metric computation later
all_tracks_history = []

for t in range(n_scans):
    # Generate noisy measurements from each target
    detections = []
    scan_truth = []
    
    for traj_fn in [fighter_trajectory, transport_trajectory, missile_trajectory]:
        truth_pos = traj_fn(t)
        scan_truth.append(truth_pos)
        noisy = truth_pos + np.random.randn(3) * 50.0  # 50m noise std
        detections.append(noisy)
    
    truths.append(scan_truth)
    
    # Process scan
    confirmed = tracker.process_scan(np.array(detections))
    all_tracks_history.append(len(confirmed))

print(f"After {n_scans} scans:")
print(f"  Total tracks created: {tracker._next_id - 1}")
print(f"  Currently confirmed: {len(tracker.confirmed_tracks)}")
for track in tracker.confirmed_tracks:
    pos = track.filter.x[:3] if hasattr(track.filter, 'x') else track.filter.filters[0].x[:3]
    print(f"  Track {track.track_id}: pos=[{pos[0]:.0f}, {pos[1]:.0f}, {pos[2]:.0f}]")

## 3. Quality Metrics — OSPA & SIAP

In [ ]:
# Get final track positions
track_positions = []
for track in tracker.confirmed_tracks:
    pos = track.filter.x[:3] if hasattr(track.filter, 'x') else track.filter.filters[0].x[:3]
    track_positions.append(pos)

truth_positions = truths[-1]  # Last scan ground truth

if track_positions:
    tp = np.array(track_positions)
    gt = np.array(truth_positions)
    
    ospa = compute_ospa(tp, gt, c=5000, p=2)
    siap = compute_siap_metrics(tracker, gt)
    
    print(f"OSPA distance: {ospa:.1f} m")
    print(f"SIAP Completeness: {siap['completeness']:.1%}")
    print(f"SIAP Purity: {siap['purity']:.1%}")
    print(f"False tracks: {siap['false_tracks']}")

## 4. Compare Association Methods

In [ ]:
results = {}
for method in ['gnn', 'jpda', 'mht']:
    np.random.seed(42)
    t = MultiTargetTracker(dt=1.0, r_std=50.0, association=method, domain="military")
    
    for scan in range(60):
        dets = []
        for fn in [fighter_trajectory, transport_trajectory, missile_trajectory]:
            dets.append(fn(scan) + np.random.randn(3) * 50.0)
        t.process_scan(np.array(dets))
    
    tp = [tr.filter.x[:3] if hasattr(tr.filter, 'x') else tr.filter.filters[0].x[:3] 
          for tr in t.confirmed_tracks]
    
    if tp:
        ospa_val = compute_ospa(np.array(tp), np.array(truths[-1]), c=5000, p=2)
    else:
        ospa_val = 5000.0
    
    results[method] = {
        'confirmed': len(t.confirmed_tracks),
        'ospa': ospa_val,
    }
    print(f"{method.upper():5s}: {len(t.confirmed_tracks)} tracks, OSPA={ospa_val:.1f}m")

print("\n✅ All three methods produce quality tracks")

## 5. Domain Presets

In [ ]:
domains = ['military', 'atc', 'automotive', 'space', 'maritime']
for domain in domains:
    dt_map = {'military': 1.0, 'atc': 4.0, 'automotive': 0.05, 'space': 10.0, 'maritime': 2.0}
    r_map = {'military': 50.0, 'atc': 100.0, 'automotive': 0.5, 'space': 200.0, 'maritime': 50.0}
    
    t = MultiTargetTracker(dt=dt_map[domain], r_std=r_map[domain], domain=domain)
    print(f"  {domain:12s}: dt={dt_map[domain]:.2f}s, r_std={r_map[domain]:.0f}m, assoc={t.association}")

print("\n✅ All domain presets configured")

## Next Steps

- **Multi-sensor fusion**: See `02_multi_sensor_fusion.ipynb`
- **Intelligence pipeline**: See `03_platform_id_ecm_intent.ipynb`  
- **Full API reference**: https://nx-mimosa.readthedocs.io
- **Commercial licensing**: mladen@nexellum.com